### What is GridSearch?
GridSearch is an optimization tool that we use when tuning hyperparameters. We define the grid of parameters that we want to search through, and we select the best combination of parameters for our data.

https://towardsdatascience.com/gridsearch-the-ultimate-machine-learning-tool-6cd5fb93d07

In [ ]:
#PRUEBA DIFERENTES PARAMETROS PARA LOS ALGORITMOS Y EL OBJETIVO ES ENCONTRAR CUAL ES EL QUE DA MEJOR ACCURACY PARA LOS DATOS.

### The “Grid” in GridSearch

![grid](grid.png)

# 1: One way

In [2]:
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning) #Esto lo pone porque hay algunas librerias que van a estar a punto de caducarse y esto es para que no salgan los warnings

In [3]:
from sklearn import svm, datasets #importamos datasets praa coger datasets
from sklearn.model_selection import GridSearchCV
iris = datasets.load_iris()

# 'kernel':('linear', 'poly', 'rbf', 'sigmoid'), 

parameters = { #Estos son datos a probar Creamos un diccionario y le ponemos lo que creemos que pruebe, en este caso linear, rbd, sgmoid
    'kernel':('linear', 'rbf', 'sigmoid'), 
    'C':[0.0001,0.1, 0.5, 1, 5, 10, 100], 
    'degree': [1,2,3,4,5,6,7,8,9],
    'coef0': [-10.,-1., 0., 0.1, 0.5, 1, 10, 100],
    'gamma': ('scale', 'auto')
    }

svc = svm.SVC() #Instanciamos la clase del modelo que queremos utilizar

clf = GridSearchCV(estimator=svc, param_grid=parameters, n_jobs=-1, cv=10)  #llamamos a grid search y le decimos que llame a ese algoritmo cv= significa q le pasamos un kfold especifico.
clf.fit(iris.data, iris.target) # Hacemos un fit y le pasamos los datos.  IRis es un dataset que tiene los datos de las flores. X = iris.data, y = iris.tardet

print("clf.best_stimator_", clf.best_estimator_)
print("clf.best_params_", clf.best_params_)
# Mean cross-validated score of the best_estimator
print("clf.best_score", clf.best_score_) #Me halla el mejor score

#el mejor que me sale es el SVC con C=0.5, y con los demas datos que esta en el print
#

clf.best_stimator_ SVC(C=0.5, coef0=-10.0, degree=1, kernel='linear')
clf.best_params_ {'C': 0.5, 'coef0': -10.0, 'degree': 1, 'gamma': 'scale', 'kernel': 'linear'}
clf.best_score 0.9866666666666667


# 2: Almost-Pro way

La forma pro es la que hace esto mismo y va recogiendo los errores de entrenamiento, de validación y tiene la capacidad de parar el proceso cuando se requiera además de guardar el modelo en local una vez terminado si es mejor que el que había anteriormente y de cargar el modelo anterior y seguir reentrenando.

In [4]:
import pickle # Para cargar una variable local

In [5]:
# Load libraries
import numpy as np
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split 
# Set random seed
np.random.seed(0)

In [6]:
# Load data
iris = datasets.load_iris()
X = iris.data
y = iris.target


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

In [8]:
to_test = np.arange(1, 10)

In [9]:
# Create a pipeline.  Pipeline a parte de probar gridsearch con un modelo sino con varios parametros que hemos definido.  El pipeline lo que va a decir es crearme una serie de pasos donde va a probar lo que pone despues de steps.  En el pipeline se cambiara el classficador a probar, es decir, los parametros de cada clasifcador (Classifier que es el algoritmo)

# Le podemos poner cualquier clasificador. Irá cambiando según va probando pero necesita 1.
pipe = Pipeline(steps=[('classifier', RandomForestClassifier())])

#Hay que poner Classifier dos barras bajas y luego el nombre del parametro.  
logistic_params = { 
    'classifier': [LogisticRegression()],
    'classifier__penalty': ['l1', 'l2'],
    'classifier__C': np.logspace(0, 4, 10)
    }

random_forest_params = {
    'classifier': [RandomForestClassifier()],
    'classifier__n_estimators': [10, 100, 1000],
    'classifier__max_features': [1, 2, 3]
    }

svm_params = {
    'classifier': [svm.SVC()],
    'classifier__kernel':('linear', 'rbf', 'sigmoid'), 
    'classifier__C':[0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], 
    'classifier__degree': to_test,
    'classifier__coef0': [-10.,-1., 0., 0.1, 0.5, 1, 10, 100],
    'classifier__gamma': ('scale', 'auto')
    }

#Para saber que parametros osn los mejores, se llama hypertuning

# Create space of candidate learning algorithms and their hyperparameters. Tenemos que crear el espacio de busuqeda, y le damos los dicccionarios a buscar. POdriamos poner mas modelos si lo necesitamos
search_space = [
    logistic_params,
    random_forest_params,
    svm_params
    ]

In [10]:
# PAra que nos de la info que nos ha tardado la ejecucion ?

cv = RepeatedKFold(n_splits=10, n_repeats=1, random_state=1) 
# Create grid search 
clf = GridSearchCV(estimator=pipe, param_grid=search_space, cv=cv, verbose=10, n_jobs=-1) #estimator= pipe, se le pasa la variable de pipe (que ya esta creado). Si queremos que pruebe parametros diferentes para varios modelo le pasamos el pipe line que se va a ocupar de probar cada algoritmo con sus parametros.  Le pasamos param grid que es la lista de parametros, lo hacemos con cv que s cross validation.

#VERBOSE = cuanto mas alto pongamos el numero de verbose, mas informacion me va a dar al hacer el print.  Las tareas que tiene que hacer es probar cada algoritmo con cada una de todas las opciones de parametros que hay es lo que ahace, y el verbose me pone cuanto esta tardando por tareas.   Esto esta bien para ver si nos hemos pasado con el numero de parametros que hemos puesto.  Me pone loq ue se tarda en x numero d tasks.  Si veo que tarda mucho para x tareas entonces alli puedo saber lo que puede tardar y yo mismo tomar la desicion.  Con esto mas o menos podemos saber lo que puede tardar si pusiera mas datos, hariamos una retgla de tres para saber.

# Fit grid search
best_model = clf.fit(X_train, y_train)

# View best model
separator = "\n############################\n"
print(separator)
print("best estimator:", best_model.best_estimator_.get_params()['classifier']) #Que me muestre el mejor estimador con el mejor modelo
print(separator)
print("clf.best_params_", clf.best_params_)
print(separator)
# Mean cross-validated score of the best_estimator
print("clf.best_score", clf.best_score_)
#SAVE MODEL
# save the model to disk
filename = 'finished_model.sav' #Guardamos el modelo una vez que ya hemos entrenado el modelo
pickle.dump(best_model, open(filename, 'wb')) #Lo guardamos en una variable

#Esta probando cada modelo con todos los parametros que hemos puesto, por eso tarda varios segundos en ejecutar

Fitting 10 folds for each of 3485 candidates, totalling 34850 fits

############################

best estimator: SVC(C=0.5, coef0=-10.0, degree=1, kernel='linear')

############################

clf.best_params_ {'classifier': SVC(C=0.5, coef0=-10.0, degree=1, kernel='linear'), 'classifier__C': 0.5, 'classifier__coef0': -10.0, 'classifier__degree': 1, 'classifier__gamma': 'scale', 'classifier__kernel': 'linear'}

############################

clf.best_score 0.9916666666666666


In [12]:
loaded_model = pickle.load(open("finished_model.sav", "rb")) #cargamos el modelo, pero si que el tenia en clases 2 lineas mas con el path

In [14]:
loaded_model

GridSearchCV(cv=RepeatedKFold(n_repeats=1, n_splits=10, random_state=1),
             estimator=Pipeline(steps=[('classifier',
                                        RandomForestClassifier())]),
             n_jobs=-1,
             param_grid=[{'classifier': [LogisticRegression()],
                          'classifier__C': array([1.00000000e+00, 2.78255940e+00, 7.74263683e+00, 2.15443469e+01,
       5.99484250e+01, 1.66810054e+02, 4.64158883e+02, 1.29154967e+03,
       3.59381366...
                          'classifier__n_estimators': [10, 100, 1000]},
                         {'classifier': [SVC(C=0.5, coef0=-10.0, degree=1,
                                             kernel='linear')],
                          'classifier__C': [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8,
                                            0.9],
                          'classifier__coef0': [-10.0, -1.0, 0.0, 0.1, 0.5, 1,
                                                10, 100],
                          'classi

In [13]:
# Predict target vector
best_model.score(X_test, y_test) * 100

100.0

# 3 Another way - No pro

In [28]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC

# Loading the Digits dataset
digits = datasets.load_digits()

# To apply an classifier on this data, we need to flatten the image, to
# turn the data in a (samples, feature) matrix:
n_samples = len(digits.images)
X = digits.images.reshape((n_samples, -1))
y = digits.target

In [29]:
# Split the dataset in two equal parts
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.5, random_state=0) #0.5 lo ponemos porque pq a medida que aumentamos el conjunto de test para probar algo, sabemos que a medida que si aumentamos el conjunto de test tambien se obtiene un buen acierto.  Si con un 50% de datos me sale que tengo un 99% de datos, es que mi modelo esta muy bien.  Siginificaria que nuestros datos para los modelos quiere decir que tiene simetria, y que se puede encontrar patrones relevantes.

# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 
                     'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},

                      {'kernel': ['linear'], 
                    'C': [1, 10, 100, 1000]}]

scores = ['precision', 'recall'] #Son metricas.. El que estamos acostrumbrado es accuracy q es el tipico score que estabamos acostumbrados creado.  Lo podemos meter en eta lista

In [34]:
bar = "######################################"
for score in scores:
    print(bar + "\n########## SCORE " + score + " ########### \n" + bar)
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(estimator=SVC(), param_grid=tuned_parameters, scoring='%f_macro'%score)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score'] #cv_results es que se está haciendo con un cross_validation especifico. y a cada uno de ellos le saco la media del test score
    stds = clf.cv_results_['std_test_score'] #Esto es la media del no se que
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred)) #Le pasamos lo que debe dar y la prediccion, nos da el reporte.  ME da un f1 score de 0.99, casi el 100%.
    print()

######################################
########## SCORE precision ########### 
######################################
# Tuning hyper-parameters for precision

Best parameters set found on development set:

{'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}

Grid scores on development set:

0.983 (+/-0.015) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.956 (+/-0.027) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.985 (+/-0.014) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.981 (+/-0.020) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.985 (+/-0.014) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.981 (+/-0.019) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.985 (+/-0.014) for {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}
0.981 (+/-0.019) for {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}
0.976 (+/-0.002) for {'C': 1, 'kernel': 'linear'}
0.976 (+/-0.002) for {'C': 10, 'kernel': 'linear'}
0.976 (+/-0.002) for {'C': 100, 'kernel': 'linear'}
0.976 (+/-0.002) for {'C': 1000, 'kernel': '